<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fabían Castro`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [1]:
#data handlign
import pandas as pd
import numpy as np

#twitter scrapper
import snscrape.modules.twitter as sntwitter

#model
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

pd.set_option('display.max_colwidth', None)

In [2]:
tweets = []
maxTweets = 2000
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('@overwatch ' + 'since:2021-01-01 until:2021-05-24 -filter:replies lang:en').get_items()) :
    if i > maxTweets :
        break
        
    temp = []
    temp.append(tweet.date)
    temp.append(tweet.content)
    
    tweets.append(temp)

In [3]:
df = pd.DataFrame(tweets)
df.columns = ['date', 'content']
display(df.sample(5))
df.shape

,date,content
23,2021-04-25 19:24:27+00:00,Wow just wow. I got kicked out twice. @overwatch https://t.co/V09nFCFQae
9,2021-05-19 00:59:36+00:00,Former @Overwatch MVP suspended https://t.co/4ONqWyYF62 #axiosgaming
117,2021-02-04 22:12:28+00:00,I'm digging this highlight into @Overwatch #PS4share\n\nhttps://t.co/QpHmyZFHRn https://t.co/kBagS2rYJm
89,2021-02-19 23:26:53+00:00,Currently drooling over @overwatch 2 watching behind the scenes of #Blizzcon2021 #overwatch2 i need this in my life asap.
107,2021-02-10 02:15:46+00:00,I forgot how much I love @overwatch. Anyone want to jam out? \n\nhttps://t.co/LWXcfvVfPR https://t.co/FxSk0ufVyS


(153, 2)

In [4]:
import re
#cleaning
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def remove_words_by_start_char(text, character):
    """Based on a starting character (i.e. @ or #),
        removes the word (with character included) from the text and returns all ocurrences"""
    spec_words = [interaction for interaction in text.split() if interaction.startswith(character)]
    for word in spec_words:
        text = text.replace(word, '')
    return text, spec_words
    
def text_clean(row):
    text = row['content']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
         
    text, hashtags = remove_words_by_start_char(text, '#')
    text, mentions = remove_words_by_start_char(text, '@')    
        
    return text#, links, hashtags, mentions

In [5]:
df['content'] = df.apply(text_clean, axis = 1)

In [6]:
df.sample(5)

,date,content
23,2021-04-25 19:24:27+00:00,Wow just wow. I got kicked out twice.
29,2021-04-19 05:46:11+00:00,"Dude what is wrong with UAE player they using hack and promoted on many games specially , Activision blizzard should band UAE also they buy Koran account and Chinese, they don’t care if they got band because they have a lots of money"
88,2021-02-20 01:20:44+00:00,can u make mccree naked
96,2021-02-16 20:35:50+00:00,"Who likes overwatch, i do. But what your favorite character on overwatch"
64,2021-03-18 06:41:26+00:00,DRUNK Raging Racists \n


In [7]:
vader = SentimentIntensityAnalyzer()
df['sent_vader'] = df['content'].apply(lambda val: vader.polarity_scores(val)['compound'])

In [8]:
df.sample(5)

,date,content,sent_vader
57,2021-03-26 18:13:33+00:00,sooo about that cozy stardew stream today... suggested we join forces for some and I've been meaning to do pachimari roadhog makeup so it's pew pew shooty time instead! time to play with pink and green 💗💚\n\n,0.9324
13,2021-05-15 15:55:57+00:00,Why do WoW players like Nightelf Women? It's those Darnassus \n \n,0.7430
138,2021-01-13 19:42:04+00:00,"The flex role has ruined a majority of comp matches in Nothing worse than playing with DPS players who have NO game sense in tank or support, and have NO intention of learning their role. So long as they get that queue time down eh?",-0.6490
76,2021-03-03 20:01:58+00:00,PODCAST GUEST THIS WEEK!! takes some unique parts and makes things just like Watson from Overwatch! LINK IN BIO! \nFOLLOW ➡️ ⬅️ for more LEGO MOCs that bring smiles 😁😁 SUBSCRIBE to the Back 2 Brick PODCAST!\n-,0.9147
98,2021-02-15 09:29:13+00:00,Lullaby for a Sadist on available on I can help 1 2 3 4 5,0.4019


###  `[25 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

### Tuit Más Negativo

In [9]:
df[['content', 'sent_vader']].sort_values('sent_vader', ascending = 1).head(1)

,content,sent_vader
64,DRUNK Raging Racists \n,-0.876


### Tuit Más Positivo

In [10]:
df[['content', 'sent_vader']].sort_values('sent_vader', ascending = 0).head(1)

,content,sent_vader
19,😇 Angelic DROP! 😇\n\nMeet Stacy Mercy! \n\nMercy is Stacys favorite character 🥰❤ \n\nI know gas ⛽ is CRAZY today but this supportive sweetie will be there for you whenever you're ready!\n\n,0.97
